In [ ]:
# Import our dependencies
!pip install keras_tuner
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN'.
application_df = application_df.drop(columns=["EIN"])

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
df = pd.DataFrame(application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts()<500])
application_types_to_replace= df.index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
df = pd.DataFrame(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()<100])
df.index.tolist()
classifications_to_replace = df.index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns="IS_SUCCESSFUL")
X.head()
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

number_input_features = len( X_train_scaled[0])


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=21, activation="tanh"))

# third hidden layer
nn_model.add(tf.keras.layers.Dense(units=26, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 117708    
                                                                 
 dense_1 (Dense)             (None, 21)                147       
                                                                 
 dense_2 (Dense)             (None, 26)                572       
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 118,454
Trainable params: 118,454
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 8s 9ms/step - loss: 0.5341 - accuracy: 0.7316
Epoch 2/20
804/804 [==============================] - 6s 7ms/step - loss: 0.1766 - accuracy: 0.9382
Epoch 3/20
804/804 [==============================] - 6s 8ms/step - loss: 0.1078 - accuracy: 0.9605
Epoch 4/20
804/804 [==============================] - 6s 7ms/step - loss: 0.0974 - accuracy: 0.9623
Epoch 5/20
804/804 [==============================] - 7s 8ms/step - loss: 0.0945 - accuracy: 0.9622
Epoch 6/20
804/804 [==============================] - 6s 7ms/step - loss: 0.0922 - accuracy: 0.9631
Epoch 7/20
804/804 [==============================] - 7s 8ms/step - loss: 0.0915 - accuracy: 0.9632
Epoch 8/20
804/804 [==============================] - 6s 8ms/step - loss: 0.0901 - accuracy: 0.9635
Epoch 9/20
804/804 [==============================] - 7s 8ms/step - loss: 0.0906 - accuracy: 0.9629
Epoch 10/20
804/804 [==============================] - 6s 8ms/step - loss: 0.0895 - accuracy: 0.9636

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4954 - accuracy: 0.7833 - 733ms/epoch - 3ms/step
Loss: 0.4954492151737213, Accuracy: 0.7833235859870911


In [ ]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")